In [1]:
# Parameters
VERSION = "9.09"
NFOLDS = 6
SEED = 3319


In [2]:
run -i import_file.py

In [3]:
modelA_cat_train = pd.read_csv('modelA_cat_data.csv')['modelA_train']
modelA_cat_test = pd.read_csv('modelA_cat_data.csv')['modelA_test']
modelA_lgbm_train = pd.read_csv('modelA_lgbm_data.csv')['modelA_lgbm_train']
modelA_lgbm_test = pd.read_csv('modelA_lgbm_data.csv')['modelA_lgbm_test']
modelB_cat_train = pd.read_csv('modelB_cat_data.csv')['modelB_train']
modelB_cat_test = pd.read_csv('modelB_cat_data.csv')['modelB_test']
modelB_lgbm_train = pd.read_csv('modelB_lgbm_data.csv')['modelB_lgbm_train']
modelB_lgbm_test = pd.read_csv('modelB_lgbm_data.csv')['modelB_lgbm_test']
modelC_cat_train = pd.read_csv('modelC_cat_data.csv')['modelC_cat_train']
modelC_cat_test = pd.read_csv('modelC_cat_data.csv')['modelC_cat_test']
modelC_lgbm_train = pd.read_csv('modelC_lgbm_data.csv')['modelC_lgbm_train']
modelC_lgbm_test = pd.read_csv('modelC_lgbm_data.csv')['modelC_lgbm_test']
modelC_ridge_train = pd.read_csv('modelC_ridge_data.csv')['modelC_ridge_train']
modelC_ridge_test = pd.read_csv('modelC_ridge_data.csv')['modelC_ridge_test']
modelD_cat_train = pd.read_csv('modelD_cat_data.csv')['modelD_cat_train']
modelD_cat_test = pd.read_csv('modelD_cat_data.csv')['modelD_cat_test']
modelD_lgbm_train = pd.read_csv('modelD_lgbm_data.csv')['modelD_lgbm_train']
modelD_lgbm_test = pd.read_csv('modelD_lgbm_data.csv')['modelD_lgbm_test']
modelE_cat_train = pd.read_csv('modelE_cat_data.csv')['modelE_cat_train']
modelE_cat_test = pd.read_csv('modelE_cat_data.csv')['modelE_cat_test']
modelE_lgbm_train = pd.read_csv('modelE_lgbm_data.csv')['modelE_lgbm_train']
modelE_lgbm_test = pd.read_csv('modelE_lgbm_data.csv')['modelE_lgbm_test']
y_train = pd.read_csv('y_train.csv', encoding='cp949').Salary
test_id = pd.read_csv('X_test.csv', encoding='cp949')['ID']

- ver1

In [4]:
X_train  = pd.concat([modelA_cat_train,modelA_lgbm_train,modelB_cat_train,modelB_lgbm_train
                      ,modelC_cat_train,modelC_lgbm_train,modelD_cat_train,modelD_lgbm_train],axis=1)
X_test  = pd.concat([modelA_cat_test,modelA_lgbm_test,modelB_cat_test,modelB_lgbm_test
                      ,modelC_cat_test,modelC_lgbm_test,modelD_cat_test,modelD_lgbm_test],axis=1).dropna()

In [5]:
model = Ridge(alpha = 1.0 )

In [6]:
from sklearn.model_selection import ShuffleSplit
#sscv = ShuffleSplit(test_size=0.3334, n_splits=5,random_state=12)
# OOF Prediction
models = cross_validate(model,
                        X_train, y_train,
                        cv=NFOLDS,
                        scoring='neg_mean_squared_error',
                        return_estimator=True)
oof_pred = np.array([m.predict(X_test) for m in models['estimator']]).mean(axis=0)

scores = models['test_score']
print("\nTuned LM CV scores: ", np.sqrt(-1*scores))
print("Tuned LM CV mean = %.2f" % np.sqrt(-1*scores.mean()), "with std = %.2f" % np.sqrt(scores.std()))


Tuned LM CV scores:  [740.66912686 725.09874575 758.62384107 779.30720906 879.36653434
 900.84344866]
Tuned LM CV mean = 800.21 with std = 333.04


In [7]:
# submission 화일 생성
LM_VERSION = VERSION

filename = f'LM_{LM_VERSION}_{np.sqrt(-1*scores.mean()):.2f}_{np.sqrt(scores.std()):.2f}.csv'
pd.DataFrame({'ID':test_id, 'Salary':oof_pred}).to_csv(filename, index=False)